In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable

open_api_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client

openai_client = OpenAI(api_key = open_api_key)



In [ ]:
# 15

from modules.pdf_extraction import select_pdf_file
from modules.pdf_extraction import extract_text_from_pages

pdf_path = select_pdf_file()
page_no = 0
extracted_text = extract_text_from_pages(pdf_path, pages=page_no)


In [ ]:
import pymupdf 
## 111
import base64
import requests

import importlib
import modules.llm
import modules.pdf_extraction


from modules.pdf_extraction import get_page_pixel_data



# this may have to happen on a page by page basis

base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                    dpi = 500, image_type = 'png')

In [4]:
# import pymupdf

# # Open some document, for example a PDF (could also be EPUB, XPS, etc.)
# doc = pymupdf.open(pdf_path)

# # Load a desired page. This works via 0-based numbers
# page = doc[0]  # this is the first page

# # Look for tables on this page and display the table count
# tabs = page.find_tables()
# num_tables = len(tabs.tables)

In [5]:
import modules.llm
importlib.reload(modules.llm)
from modules.llm import table_identification_llm

output_table_identification_llm = await table_identification_llm(text_input=extracted_text,  base64_image=base64_image, open_api_key=open_api_key)


In [ ]:
output_table_identification_llm

In [ ]:
importlib.reload(modules.pdf_extraction)
from modules.pdf_extraction import get_validated_table_info
num_tables, table_headers, table_location, confidence_score_0 = await get_validated_table_info(text_input=extracted_text, open_api_key=open_api_key, base64_image= base64_image)

In [ ]:
# from modules.pdf_extraction import table_identification_llm


# output = await table_identification_llm(text_input=extracted_text, base64_image=base64_image, open_api_key=open_api_key)


In [12]:
if num_tables == 0:
    raise ValueError("No tables found on the page")



In [9]:
# add in control for no tables found
# table_with_index = " || ".join([f"index {i}: {header}" for i, header in enumerate(table_headers)])

In [10]:
# tables_data = [f"[{i}]" for i in table_headers]

In [ ]:
extracted_text

In [ ]:
import json
import pandas as pd
import re
import importlib
import modules.pdf_extraction
importlib.reload(modules.pdf_extraction)
from modules.pdf_extraction import process_tables_to_df


user_input = "I would like to get all the data text from the table"

output_1 = await process_tables_to_df(
                     user_text=user_input,
                     table_location = table_location,
                     table_headers = table_headers,
                     extracted_text=extracted_text, 
                     base64_image=base64_image, 
                     open_api_key=open_api_key)


output_1

In [12]:
# # add in try and except for error handling

# import json
# import pandas as pd
# import re
# import importlib
# import modules.llm
# importlib.reload(modules.llm)
# from modules.llm import vision_llm_parser

# user_input = "I would like to get all the data text from the table"

# output = await vision_llm_parser(
#                 user_text=user_input,
#                 text_input=extracted_text,
#                 table_to_target=table_headers[1],
#                 base64_image=base64_image,
#                 open_api_key=open_api_key,
#                 model='gpt-4o'
#             )

# # from modules.pdf_extraction import parse_variable_data_to_df
# # df_0 = parse_variable_data_to_df(output)
# # df_0

# # def extract_list_from_string(text):
# #     # Find the content between the first [ and last ], including the brackets
# #     match = re.search(r'(\[.*\])', text, re.DOTALL)
# #     if match:
# #         # Return the full string including brackets
# #         return match.group(1)
# #     return None

# # extracted_dict = extract_list_from_string(output)


# # data = json.loads(extracted_dict)
# # df = pd.DataFrame(data)
# # df

In [ ]:
# pdf_path = select_pdf_file()

In [ ]:
# import pymupdf

# # Open some document, for example a PDF (could also be EPUB, XPS, etc.)
# doc = pymupdf.open(pdf_path)

# # Load a desired page. This works via 0-based numbers
# page = doc[0]  # this is the first page

# # Look for tables on this page and display the table count
# tabs = page.find_tables()
# print(f"{len(tabs.tables)} table(s) on {page}")

# # We will see a message like "1 table(s) on page 0 of input.pdf"

In [ ]:
################

In [2]:
from modules.pdf_extraction import select_pdf_file

from modules.pdf_extraction import get_validated_table_info
from modules.pdf_extraction import get_page_pixel_data
from modules.pdf_extraction import process_tables_to_df
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import asyncio
import os
import pymupdf
import time



# to do.. table names headers can be dupilcated may be helpful to use say table 1,2, etc to deitigusih tem. 
# use pdf plumber  page.find_tables() and gpt table count for confidence calculation. 
# the same table can be extraected twice or more if the header is not clear
# extract header witout the text 

file_name = 'test_7'    
user_text='Extract all data from the table(s) the header'

# 1. Load Credientials

# Load environment variables from the .env file
load_dotenv()
# Get the API key from the environment variable
open_api_key = os.getenv('OPENAI_API_KEY')
# Initialize OpenAI client
openai_client = OpenAI(api_key = open_api_key)





# 2. Select PDF file and extract text
pdf_path = select_pdf_file()
doc = pymupdf.open(pdf_path)
total_pages = doc.page_count  # total number of pages in the document
page_indices = range(total_pages)

# page_indices can be a list of page numbers to process


# Start timing
start_time = time.time()

async def process_page():
    tasks = []
    results_output = []
    # Create all tasks first 
    async with asyncio.TaskGroup() as tg:
        for page_no in page_indices:
            page = doc.load_page(page_no)
            extracted_text = page.get_text()
            
            # extracted_text = extract_text_from_pages(pdf_path, pages=page_no)
            base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                                dpi = 500, image_type = 'png')
        
            num_tables, table_headers, table_location, confidence_score_0 = await get_validated_table_info(
                text_input=extracted_text, 
                open_api_key=open_api_key, 
                base64_image=base64_image
            )

            if num_tables == 0:
                print(f"No tables found on page {page_no + 1}, skipping...")
                continue
    
            tasks.append(tg.create_task(process_tables_to_df(
                table_headers, 
                table_location,
                user_text, 
                extracted_text, 
                base64_image, 
                open_api_key,
                page_number=page_no)))
            
        # Await all tasks to complete
        for task in tasks:
            results_output.append(await task)
    
    if not results_output:
        raise ValueError("No tables found on any of the processed pages")
            
    # df_out_1 = pd.concat(results_output, ignore_index=True)
    return results_output


output_final = await process_page()


2025-02-28 22:03:40,844 - INFO - Opening file selection dialog.
2025-02-28 22:03:45,385 - INFO - Selected PDF file: C:/Users/derri/Desktop/extracted 1 and 2.pdf
2025-02-28 22:03:45,425 - INFO - Converting PDF page 1 to base64 image. DPI=500, Format=png
2025-02-28 22:03:47,809 - INFO - Finished converting page to base64.
2025-02-28 22:03:47,815 - INFO - Validating table information with multiple LLM calls.
2025-02-28 22:04:05,552 - INFO - Majority match found with first and third results.
2025-02-28 22:04:05,561 - INFO - Converting PDF page 2 to base64 image. DPI=500, Format=png
2025-02-28 22:04:07,586 - INFO - Finished converting page to base64.
2025-02-28 22:04:07,591 - INFO - Validating table information with multiple LLM calls.
2025-02-28 22:04:07,592 - INFO - Processing tables to DataFrame for page 1
2025-02-28 22:04:18,603 - INFO - Initial table info match or same table count. Returning first attempt's result.
2025-02-28 22:04:18,604 - INFO - Processing tables to DataFrame for pag

In [ ]:
from modules.pdf_extraction import write_output_final
write_output_final(output_final, excel_path='files/AllTables12.xlsx', option=2, gap_rows=2)

In [53]:

# import itertools
# all_dfs = list(itertools.chain.from_iterable(output_final))
# combined_dfs = pd.concat(all_dfs)


# combined_dfs['descriptor_label'] = (
#     combined_dfs.groupby('page_number')['table_header_descriptor']
#     .transform(lambda x: pd.factorize(x)[0] + 1)
# )


# combined_dfs['page_table'] = (
#     'page ' + combined_dfs['page_number'].astype(str) +
#     ' table ' + combined_dfs['descriptor_label'].astype(str)
# )


In [ ]:
# mapping_dict = combined_dfs.groupby('table_header_descriptor')['page_table'].apply(list).to_dict()
# mapping_dict


In [62]:
# df['table_header_descriptor'].map(mapping_dict)

In [36]:
# import itertools


# all_dfs = list(itertools.chain.from_iterable(output_final))
# combined_dfs = pd.concat(all_dfs)
# combined_dfs['table_header_descriptor_factorized'], _ = pd.factorize(combined_dfs['table_header_descriptor'])



# unique_headers = combined_dfs['table_header_descriptor_factorized'].unique()
# header_to_num = {
#     header: f"Page {combined_dfs['page_number'] + 1} Table {i+1}"
#     for i, header in enumerate(unique_headers)
# }

In [10]:
# # Generate a "page_table" label using unique descriptors
# unique_headers = df['table_header_descriptor'].unique()
# header_to_num = {
#     header: f"Page {df['page_number'] + 1} Table {i+1}"
#     for i, header in enumerate(unique_headers)
# }
# df['page_table'] = df['table_header_descriptor'].map(header_to_num)

# # df.drop(columns=['table_header_descriptor'], inplace=True)


In [11]:
# write combined tables to excel

df_combined = pd.concat(output_final[0], ignore_index=True)
df_combined.to_excel(f'files/{file_name}.xlsx', index=False)


In [ ]:
from modules.pdf_extraction import write_split_tables_to_excel

# Call the function with output_final[0]
write_split_tables_to_excel(output_final[0],output_path='files/test_output.xlsx')
